In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

In [2]:
data = pd.read_csv('final_frame_pl_v4.csv')

In [3]:
data['FTR'] = data['FTR'].replace({'H': 1, 'A': 0, 'D':0.5}).astype(float)  

In [4]:
data = data.drop(columns = ['B365H', 'B365D', 'B365A', 'Bet_diff', 'Bet_diff_bin', 'Bet_diff_std', 'Bet_winner'])

In [5]:
data = data.dropna()

In [6]:
data_1 = data[data['Game_settled'] == 1]  
data_0 = data[data['Game_settled'] == 0]  

data_1_sample = data_1.sample(n=len(data_0), random_state=42)

balanced_data = pd.concat([data_1_sample, data_0])

balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)

In [7]:
balanced_data

Date          HomeTeam  ... Form_winner  Index
0     2019-12-21 00:00:00           Everton  ...           1   8150
1     2018-01-09 00:00:00          Brighton  ...           0   7630
2     2020-08-07 00:00:00          West Ham  ...           0   8315
3     2014-12-15 00:00:00           Everton  ...           0   5479
4     1997-11-01 00:00:00         Wimbledon  ...           1  10094
...                   ...               ...  ...         ...    ...
5397  2006-08-22 00:00:00           Watford  ...           0   2291
5398  2021-12-12 00:00:00           Burnley  ...           0   7375
5399  2022-04-24 00:00:00          Brighton  ...           1   7546
5400  2024-07-04 00:00:00  Sheffield United  ...           0   9051
5401  2015-09-26 00:00:00        Man United  ...           1   5762

[5402 rows x 42 columns]

In [8]:
features = ['Pos_diff_std', 'Avg_pts_diff_std', 'Avg_goals_scored_diff_std', 'Form_diff_std', 'Draw_ratio_diff_std']
X = data[features]  
y = data['Game_settled']   

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7479030754892824
              precision    recall  f1-score   support

         0.0       0.35      0.06      0.10       515
         1.0       0.76      0.97      0.85      1631

    accuracy                           0.75      2146
   macro avg       0.56      0.51      0.48      2146
weighted avg       0.66      0.75      0.67      2146



In [9]:
data.shape

(10727, 42)

In [10]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train, y_train)

y_pred_xgb = xgb_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))

Accuracy: 0.739049394221808
              precision    recall  f1-score   support

         0.0       0.34      0.10      0.15       515
         1.0       0.77      0.94      0.85      1631

    accuracy                           0.74      2146
   macro avg       0.56      0.52      0.50      2146
weighted avg       0.67      0.74      0.68      2146



In [11]:
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = RandomizedSearchCV(RandomForestClassifier(random_state=42), param_grid, n_iter=10, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

In [12]:
# version 1.2
best_model.fit(X_train, y_train)

y_pred_best = best_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

Accuracy: 0.7600186393289842
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       515
         1.0       0.76      1.00      0.86      1631

    accuracy                           0.76      2146
   macro avg       0.38      0.50      0.43      2146
weighted avg       0.58      0.76      0.66      2146



In [13]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

ensemble_model = VotingClassifier(estimators=[
    ('rf', best_model),
    ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000))
], voting='soft')

ensemble_model.fit(X_train, y_train)
y_pred_ens_mod = ensemble_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_ens_mod))
print(classification_report(y_test, y_pred_ens_mod))

Accuracy: 0.7600186393289842
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       515
         1.0       0.76      1.00      0.86      1631

    accuracy                           0.76      2146
   macro avg       0.38      0.50      0.43      2146
weighted avg       0.58      0.76      0.66      2146

